# Zadanie
## Autori: Rajcsanyi Roland, Setnicky Jakub Frantisek

Fáza 3: Strojové učenie (max. 18b)
Pri dátovej analýze nemusí byť naším cieľom získať len znalosti obsiahnuté v aktuálnych dátach, ale aj natrénovať model, ktorý bude schopný robiť rozumné predikcie pre nové pozorovania pomocou techniky strojového učenia.

V tomto projekte sa zameriame na rozhodovacie stromy vzhľadom na ich jednoduchú interpretovateľnosť. V tejto fáze použite všetky data, ktoré ste dostali. Oddemonštrujete znovupoužiteľnosť vami realizovaného predspracovania.

Predspracovanie validačného datasetu Vami realizovaným postupom predspracovania a opis prípadných zmien (3b).

Spustite postup predspracovania realizovaný v predchádzajúcej fáze nad novým datasetom. Nový dataset bude mať rovnakú štruktúru ako váš pôvodný.
Ak si spustenie predspracovania vyžiada zmeny v kóde, opíšte ich. Zabezpečíte aby postup predspracovanie funguje rovnako na obidve datasety.
Manuálne vytvorenie a vyhodnotenie rozhodovacích pravidiel pre klasifikáciu (3b).

Vyskúšajte jednoduché pravidlá (1R) zahŕňajúce jeden atribút pre CART, ale aj komplikovanejšie zahŕňajúce viacero atribútov (ich kombinácie).
Pravidlá by v tomto kroku mali byť vytvorené manuálne na základe pozorovaných závislostí v dátach.
Pravidlá (manuálne vytvorené klasifikátory) vyhodnoťte pomocou metrík accuracy, precision a recall.
Natrénovanie a vyhodnotenie klasifikátora s využitím rozhodovacích stromov (6b).

Na trénovanie využite stromový algoritmus (resp. algoritmy) dostupný/é v scikit-learn.
Vizualizujte natrénované pravidlá.
Vyhodnoťte natrénovaný rozhodovací strom pomocou metrík accuracy, precision a recall
Porovnajte natrénovaný klasifikátor s Vašimi manuálne vytvorenými pravidlami z druhého kroku.
Optimalizácia hyperparametrov (4b).

Preskúmajte hyperparametre Vášho zvoleného klasifikačného algoritmu a vyskúšajte ich rôzne nastavenie tak, aby ste minimalizovali overfitting (preučenie) a optimalizovali výsledok. Vysvetlite, čo jednotlivé hyperparametre robia.
Pri nastavovaní hyperparametrov algoritmu využite 10-násobnú krížovú validáciu na trénovacej množine.
Vyhodnotenie vplyvu zvolenej stratégie riešenia na správnosť klasifikácie (2b).

Zistite, či použitie zvolených stratégií riešenia chýbajúcich hodnôt vplýva na accuracy klasifikácie.
Ktorá stratégia sa ukázala ako vhodnejšia pre daný problém?

In [ ]:
import pandas as pd
import numpy as np
import statistics as stat
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
import math

nacitanie datasetov

In [ ]:
valid_SI_dataset = pd.read_csv("Dataset/valid_SI_processed_dataset.csv")
train_SI_dataset = pd.read_csv("Dataset/train_SI_processed_dataset.csv")
valid_KNI_dataset = pd.read_csv("Dataset/valid_KNI_processed_dataset.csv")
train_KNI_dataset = pd.read_csv("Dataset/train_KNI_processed_dataset.csv")
non_processed_valid = pd.read_csv("Dataset/other_valid.csv")
non_processed_train = pd.read_csv("Dataset/other_train.csv")

transofrmacia atributu class z float na int kvoli neskorsim metodam ktore by nepracovali spravne s float pre class

In [ ]:
valid_SI_dataset['class'] = valid_SI_dataset['class'].apply(lambda row: math.floor(row))
train_SI_dataset['class'] = train_SI_dataset['class'].apply(lambda row: math.floor(row))
valid_KNI_dataset['class'] = valid_KNI_dataset['class'].apply(lambda row: math.floor(row))
train_KNI_dataset['class'] = train_KNI_dataset['class'].apply(lambda row: math.floor(row))

# Porovnanie predspracovania

Nizsie su vypisane atributy predspracovanych validacnych ako aj trenovacich datasetov. Vidime ze predspracovanie vybralo rovnake atributy z oboch datasetov. Pocet zaznamov bol v datasetoch rozdielny a tak je rozdielny aj pocet zaznamov po spracovani. Rozdiel v pocetnosti zaznamov sposobil ze sa rozdelenia atributov mierne odlisuju medzi datasetmi ked zoberieme do uvahy pomer rozdeleni pred a po spracovani avsak tento rozdiel ja marginalny. To ze sa predspracovany trenovaci a validacny dataset velmi podobaju nam overilo ze predspracovanie je mozne vhodne pouzit na oba datassety a nemusime sa obavat roznych vysledkov pri pouziti hociktoreho z datasetov. Dalej vidime ze niektore atributy sa lisia v zalezitosti od toho akym sposobom boli nahradzovane NaN hodnoty a vyhodnotenie tychto dvoch rozlicnych strategii bude urobene neskor. vidime vsak ze zmena medzi strategiami je konzistentna pri validacnom aj trenovacom datasete.

In [ ]:
valid_SI_dataset.info()

In [ ]:
train_SI_dataset.info()

In [ ]:
valid_KNI_dataset.info()

In [ ]:
train_KNI_dataset.info()

In [ ]:
non_processed_valid['kurtosis_oxygen'].describe()

In [ ]:
non_processed_train['kurtosis_oxygen'].describe()

In [ ]:
valid_SI_dataset['kurtosis_oxygen'].describe()

In [ ]:
train_SI_dataset['kurtosis_oxygen'].describe()

In [ ]:
valid_KNI_dataset['kurtosis_oxygen'].describe()

In [ ]:
train_KNI_dataset['kurtosis_oxygen'].describe()

# Detekcia Overfittingu

V tejto casti vyskusame ci nie je nas model pretrenovany a pokusime sa ho zlepsit

In [ ]:
X_SI_valid = valid_SI_dataset.drop("class", axis=1)
y_SI_valid = valid_SI_dataset["class"]
X_KNI_valid = valid_KNI_dataset.drop("class", axis=1)
y_KNI_valid = valid_KNI_dataset["class"]
X_SI_train = train_SI_dataset.drop("class", axis=1)
y_SI_train = train_SI_dataset["class"]
X_KNI_train = train_KNI_dataset.drop("class", axis=1)
y_KNI_train = train_KNI_dataset["class"]

In [ ]:
X_SI_valid_train, X_SI_valid_test, y_SI_valid_train, y_SI_valid_test = train_test_split(X_SI_valid, y_SI_valid, test_size=0.33, random_state=40)
clf_SI_valid = LogisticRegression(penalty='l2', 
                         C=0.1, 
                         max_iter=5000)
clf_SI_valid.fit(X_SI_valid_train, y_SI_valid_train)

y_SI_valid_pred = clf_SI_valid.predict(X_SI_valid_test)
cm_SI_valid = metrics.confusion_matrix(y_SI_valid_test, y_SI_valid_pred)
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')


print('Logistic regression Accuracy:', metrics.accuracy_score(y_SI_valid_test, y_SI_valid_pred))
print("Logistic regresion:")
print(pd.DataFrame(cm_SI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')


y_SI_valid_pred_proba = clf_SI_valid.predict_proba(X_SI_valid_test)[::,1]
fpr_SI_valid, tpr_SI_valid, threshold_SI_valid = metrics.roc_curve(y_SI_valid_test, y_SI_valid_pred_proba)

auc_SI_valid = metrics.roc_auc_score(y_SI_valid_test, y_SI_valid_pred_proba)
print('FPR', fpr_SI_valid, 
      '\nTPR', tpr_SI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_SI_valid)

plt.plot(fpr_SI_valid, tpr_SI_valid, label="data 1, auc="+str(auc_SI_valid) )
plt.legend(loc=4)
plt.show()

In [ ]:
results_SI_valid = []
estimators_SI_valid = []
for i in range(1, X_SI_valid_train.shape[1] + 1):  
    row_SI_valid = {'model_complexity': i}
    
    # Vytvoríme rozhodovací strom
    # strom s maximalnou hlbkou 1-pocet atributov,  simulujeme tak zlozitost modelu
    clf_SI_valid = DecisionTreeClassifier(max_depth = i) 
    
    # natrenovanie modelu a predikovanie na trenovacej sade
    pred_SI_valid = clf_SI_valid.fit(X_SI_valid_train, y_SI_valid_train).predict(X_SI_valid_train) 
    
    # chyba na trenovacej sade
    row_SI_valid['train'] = 1-accuracy_score(y_SI_valid_train, pred_SI_valid) 
    
    # predickcia
    pred_SI_valid = clf_SI_valid.predict(X_SI_valid_test)
    
    # chyba na testovacej sade
    row_SI_valid['test'] = 1-accuracy_score(y_SI_valid_test, pred_SI_valid) 
    results_SI_valid.append(row_SI_valid)
    estimators_SI_valid.append(clf_SI_valid)

In [ ]:
complexity_df_SI_valid = pd.DataFrame(results_SI_valid)
complexity_df_SI_valid.head(15)

In [ ]:
complexity_df_SI_valid.plot(x='model_complexity')

In [ ]:
X_KNI_valid_train, X_KNI_valid_test, y_KNI_valid_train, y_KNI_valid_test = train_test_split(X_KNI_valid, y_KNI_valid, test_size=0.33, random_state=40)
clf_KNI_valid = LogisticRegression(penalty='l2', 
                         C=0.1, 
                         max_iter=5000)
clf_KNI_valid.fit(X_KNI_valid_train, y_KNI_valid_train)

y_KNI_valid_pred = clf_KNI_valid.predict(X_KNI_valid_test)
cm_KNI_valid = metrics.confusion_matrix(y_KNI_valid_test, y_KNI_valid_pred)
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


print('Logistic regression Accuracy:', metrics.accuracy_score(y_KNI_valid_test, y_KNI_valid_pred))
print("Logistic regresion:")
print(pd.DataFrame(cm_KNI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


y_KNI_valid_pred_proba = clf_KNI_valid.predict_proba(X_KNI_valid_test)[::,1]
fpr_KNI_valid, tpr_KNI_valid, threshold_KNI_valid = metrics.roc_curve(y_KNI_valid_test, y_KNI_valid_pred_proba)

auc_KNI_valid = metrics.roc_auc_score(y_KNI_valid_test, y_KNI_valid_pred_proba)
print('FPR', fpr_KNI_valid, 
      '\nTPR', tpr_KNI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_KNI_valid)

plt.plot(fpr_KNI_valid, tpr_KNI_valid, label="data 1, auc="+str(auc_KNI_valid) )
plt.legend(loc=4)
plt.show()

In [ ]:
results_KNI_valid = []
estimators_KNI_valid = []
for i in range(1, X_KNI_valid_train.shape[1] + 1):  
    row_KNI_valid = {'model_complexity': i}
    
    # Vytvoríme rozhodovací strom
    # strom s maximalnou hlbkou 1-pocet atributov,  simulujeme tak zlozitost modelu
    clf_KNI_valid = DecisionTreeClassifier(max_depth = i) 
    
    # natrenovanie modelu a predikovanie na trenovacej sade
    pred_KNI_valid = clf_KNI_valid.fit(X_KNI_valid_train, y_KNI_valid_train).predict(X_KNI_valid_train) 
    
    # chyba na trenovacej sade
    row_KNI_valid['train'] = 1-accuracy_score(y_KNI_valid_train, pred_KNI_valid) 
    
    # predickcia
    pred_KNI_valid = clf_KNI_valid.predict(X_KNI_valid_test)
    
    # chyba na testovacej sade
    row_KNI_valid['test'] = 1-accuracy_score(y_KNI_valid_test, pred_KNI_valid) 
    results_KNI_valid.append(row_KNI_valid)
    estimators_KNI_valid.append(clf_KNI_valid)

In [ ]:
complexity_df_KNI_valid = pd.DataFrame(results_KNI_valid)
complexity_df_KNI_valid.head(15)

In [ ]:
complexity_df_KNI_valid.plot(x='model_complexity')

Mozeme vidiet ze validacne datasety s oboma strategiami NaN maju zvysujucu sa chybu so zvysujucou sa komplexitou co znamena ze model bol pretrenovany a natrenovali sa data a nie spojenia medzi datami. skusime aplikovat Redukciu atributov pomocou Sequential forward selection na vybratie k najlepsich atributov pokym nedosiahneme klesajucu chybu

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression


In [ ]:
# Sequential Forward Selection(sfs) for SI_valid
sfs = SFS(LinearRegression(),
          k_features=3,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)
sfs.fit(X_SI_valid, y_SI_valid)
X_SI_valid = X_SI_valid.filter(sfs.k_feature_names_)
X_SI_valid.info()

In [ ]:
X_SI_valid_train, X_SI_valid_test, y_SI_valid_train, y_SI_valid_test = train_test_split(X_SI_valid, y_SI_valid, test_size=0.33, random_state=40)
clf_SI_valid = LogisticRegression(penalty='l2', 
                         C=0.1, 
                         max_iter=5000)
clf_SI_valid.fit(X_SI_valid_train, y_SI_valid_train)

y_SI_valid_pred = clf_SI_valid.predict(X_SI_valid_test)
cm_SI_valid = metrics.confusion_matrix(y_SI_valid_test, y_SI_valid_pred)
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')


print('Logistic regression Accuracy:', metrics.accuracy_score(y_SI_valid_test, y_SI_valid_pred))
print("Logistic regresion:")
print(pd.DataFrame(cm_SI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')


y_SI_valid_pred_proba = clf_SI_valid.predict_proba(X_SI_valid_test)[::,1]
fpr_SI_valid, tpr_SI_valid, threshold_SI_valid = metrics.roc_curve(y_SI_valid_test, y_SI_valid_pred_proba)

auc_SI_valid = metrics.roc_auc_score(y_SI_valid_test, y_SI_valid_pred_proba)
print('FPR', fpr_SI_valid, 
      '\nTPR', tpr_SI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_SI_valid)

plt.plot(fpr_SI_valid, tpr_SI_valid, label="data 1, auc="+str(auc_SI_valid) )
plt.legend(loc=4)
plt.show()

In [ ]:
results_SI_valid = []
estimators_SI_valid = []
for i in range(1, X_SI_valid_train.shape[1] + 1):  
    row_SI_valid = {'model_complexity': i}
    
    # Vytvoríme rozhodovací strom
    # strom s maximalnou hlbkou 1-pocet atributov,  simulujeme tak zlozitost modelu
    clf_SI_valid = DecisionTreeClassifier(max_depth = i) 
    
    # natrenovanie modelu a predikovanie na trenovacej sade
    pred_SI_valid = clf_SI_valid.fit(X_SI_valid_train, y_SI_valid_train).predict(X_SI_valid_train) 
    
    # chyba na trenovacej sade
    row_SI_valid['train'] = 1-accuracy_score(y_SI_valid_train, pred_SI_valid) 
    
    # predickcia
    pred_SI_valid = clf_SI_valid.predict(X_SI_valid_test)
    
    # chyba na testovacej sade
    row_SI_valid['test'] = 1-accuracy_score(y_SI_valid_test, pred_SI_valid) 
    results_SI_valid.append(row_SI_valid)
    estimators_SI_valid.append(clf_SI_valid)

In [ ]:
complexity_df_SI_valid = pd.DataFrame(results_SI_valid)
complexity_df_SI_valid.head(15)

In [ ]:
complexity_df_SI_valid.plot(x='model_complexity')

In [ ]:
# Sequential Forward Selection(sfs) for KNI_valid
sfs = SFS(LinearRegression(),
          k_features=4,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)
sfs.fit(X_KNI_valid, y_KNI_valid)
X_KNI_valid = X_KNI_valid.filter(sfs.k_feature_names_)
X_KNI_valid.info()

In [ ]:
X_KNI_valid_train, X_KNI_valid_test, y_KNI_valid_train, y_KNI_valid_test = train_test_split(X_KNI_valid, y_KNI_valid, test_size=0.33, random_state=40)
clf_KNI_valid = LogisticRegression(penalty='l2', 
                         C=0.1, 
                         max_iter=5000)
clf_KNI_valid.fit(X_KNI_valid_train, y_KNI_valid_train)

y_KNI_valid_pred = clf_KNI_valid.predict(X_KNI_valid_test)
cm_KNI_valid = metrics.confusion_matrix(y_KNI_valid_test, y_KNI_valid_pred)
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


print('Logistic regression Accuracy:', metrics.accuracy_score(y_KNI_valid_test, y_KNI_valid_pred))
print("Logistic regresion:")
print(pd.DataFrame(cm_KNI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


y_KNI_valid_pred_proba = clf_KNI_valid.predict_proba(X_KNI_valid_test)[::,1]
fpr_KNI_valid, tpr_KNI_valid, threshold_KNI_valid = metrics.roc_curve(y_KNI_valid_test, y_KNI_valid_pred_proba)

auc_KNI_valid = metrics.roc_auc_score(y_KNI_valid_test, y_KNI_valid_pred_proba)
print('FPR', fpr_KNI_valid, 
      '\nTPR', tpr_KNI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_KNI_valid)

plt.plot(fpr_KNI_valid, tpr_KNI_valid, label="data 1, auc="+str(auc_KNI_valid) )
plt.legend(loc=4)
plt.show()

In [ ]:
results_KNI_valid = []
estimators_KNI_valid = []
for i in range(1, X_KNI_valid_train.shape[1] + 1):  
    row_KNI_valid = {'model_complexity': i}
    
    # Vytvoríme rozhodovací strom
    # strom s maximalnou hlbkou 1-pocet atributov,  simulujeme tak zlozitost modelu
    clf_KNI_valid = DecisionTreeClassifier(max_depth = i) 
    
    # natrenovanie modelu a predikovanie na trenovacej sade
    pred_KNI_valid = clf_KNI_valid.fit(X_KNI_valid_train, y_KNI_valid_train).predict(X_KNI_valid_train) 
    
    # chyba na trenovacej sade
    row_KNI_valid['train'] = 1-accuracy_score(y_KNI_valid_train, pred_KNI_valid) 
    
    # predickcia
    pred_KNI_valid = clf_KNI_valid.predict(X_KNI_valid_test)
    
    # chyba na testovacej sade
    row_KNI_valid['test'] = 1-accuracy_score(y_KNI_valid_test, pred_KNI_valid) 
    results_KNI_valid.append(row_KNI_valid)
    estimators_KNI_valid.append(clf_KNI_valid)

In [ ]:
complexity_df_KNI_valid = pd.DataFrame(results_KNI_valid)
complexity_df_KNI_valid.head(15)

In [ ]:
complexity_df_KNI_valid.plot(x='model_complexity')

Vyssie vidime ze pri datasete SI sme museli atributy zredukovat az na 3 a pri KNI na 4 atributy. presnost modelu sa mierne znizila pri porovnani s predoslym to vsak moze byt dosledkom zle zvolenych hyperparametrov ktore budeme optimalizovat nizsie

In [ ]:
# Vizualizacia stromu pre SI_valid
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG

graph = Source(export_graphviz(estimators_SI_valid[-1], # najzlozitejsi model
                               out_file=None,
                               feature_names=X_SI_valid.columns,
                               class_names=["positive", "negative"],
                               filled = True))
display(SVG(graph.pipe(format='svg')))

from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

In [ ]:
# Vizualizacia stromu pre KNI_valid
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG

graph = Source(export_graphviz(estimators_KNI_valid[-1], # najzlozitejsi model
                               out_file=None,
                               feature_names=X_KNI_valid.columns,
                               class_names=["positive", "negative"],
                               filled = True))
display(SVG(graph.pipe(format='svg')))

from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

# Vytvorenie rozhodovacich stromov s vlastnymi pravidlami

na Zaklade pozorovani zavislosti medzi datami vytvorime rozhodovacie stromy s vlastnymi pravidlami a porovname vysledky so stromami z kniznice scikit

In [ ]:
sns.pairplot(X_KNI_valid)

In [ ]:
sns.pairplot(X_SI_valid)

In [ ]:
sns.scatterplot(x=X_KNI_valid[X_KNI_valid.columns[0]], y=y_KNI_valid)

In [ ]:
sns.scatterplot(x=X_KNI_valid[X_KNI_valid.columns[1]], y=y_KNI_valid)

In [ ]:
sns.scatterplot(x=X_KNI_valid[X_KNI_valid.columns[2]], y=y_KNI_valid)

In [ ]:
sns.scatterplot(x=X_KNI_valid[X_KNI_valid.columns[3]], y=y_KNI_valid)

In [ ]:
sns.scatterplot(x=X_SI_valid[X_SI_valid.columns[0]], y=y_SI_valid)

In [ ]:
sns.scatterplot(x=X_SI_valid[X_SI_valid.columns[1]], y=y_SI_valid)

In [ ]:
sns.scatterplot(x=X_SI_valid[X_SI_valid.columns[2]], y=y_SI_valid)

In [ ]:
class Node:
    """A decision tree node.
    code from : https://github.com/joachimvalente/decision-tree-cart/blob/master/tree.py
    used for the printing logic and basic structure of node
    """

    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

    def debug(self, feature_names, class_names, show_details):
        """Print an ASCII visualization of the tree."""
        lines, _, _, _ = self._debug_aux(
            feature_names, class_names, show_details, root=True
        )
        for line in lines:
            print(line)

    def _debug_aux(self, feature_names, class_names, show_details, root=False):
        # See https://stackoverflow.com/a/54074933/1143396 for similar code.
        is_leaf = not self.right
        if is_leaf:
            lines = [class_names[self.predicted_class]]
        else:
            lines = [
                "{} < {:.2f}".format(feature_names[self.feature_index], self.threshold)
            ]
        if show_details:
            lines += [
                "gini = {:.2f}".format(self.gini),
                "samples = {}".format(self.num_samples),
                str(self.num_samples_per_class),
            ]
        width = max(len(line) for line in lines)
        height = len(lines)
        if is_leaf:
            lines = ["║ {:^{width}} ║".format(line, width=width) for line in lines]
            lines.insert(0, "╔" + "═" * (width + 2) + "╗")
            lines.append("╚" + "═" * (width + 2) + "╝")
        else:
            lines = ["│ {:^{width}} │".format(line, width=width) for line in lines]
            lines.insert(0, "┌" + "─" * (width + 2) + "┐")
            lines.append("└" + "─" * (width + 2) + "┘")
            lines[-2] = "┤" + lines[-2][1:-1] + "├"
        width += 4  # for padding

        if is_leaf:
            middle = width // 2
            lines[0] = lines[0][:middle] + "╧" + lines[0][middle + 1 :]
            return lines, width, height, middle

        # If not a leaf, must have two children.
        left, n, p, x = self.left._debug_aux(feature_names, class_names, show_details)
        right, m, q, y = self.right._debug_aux(feature_names, class_names, show_details)
        top_lines = [n * " " + line + m * " " for line in lines[:-2]]
        # fmt: off
        middle_line = x * " " + "┌" + (n - x - 1) * "─" + lines[-2] + y * "─" + "┐" + (m - y - 1) * " "
        bottom_line = x * " " + "│" + (n - x - 1) * " " + lines[-1] + y * " " + "│" + (m - y - 1) * " "
        # fmt: on
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = (
            top_lines
            + [middle_line, bottom_line]
            + [a + width * " " + b for a, b in zipped_lines]
        )
        middle = n + width // 2
        if not root:
            lines[0] = lines[0][:middle] + "┴" + lines[0][middle + 1 :]
        return lines, n + m + width, max(p, q) + 2 + len(top_lines), middle

In [ ]:
class OwnTreeImplementation:
    def __init__(self, max_depth=None, min_split=1, min_leaf=1):
        self.max_depth = max_depth
        self.min_split = min_split
        self.min_leaf = min_leaf

    def fit(self, X, y):
        """Build decision tree classifier."""
        self.n_of_classes_ = len(set(y))  # classes are assumed to go from 0 to n-1
        self.n_of_features_ = len(X.columns)
        self.tree_full = self._grow_tree(X, y)

    def predict(self, X):
        """Predict class for X."""
        return [self._predict(inputs, X.columns) for index, inputs in X.iterrows()]

    def debug(self, feature_names, class_names, show_details=True):
        """Print ASCII visualization of decision tree."""
        self.tree_full.debug(feature_names, class_names, show_details)

    def _gini(self, y):
        m = y.size
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in range(self.n_of_classes_))

    def _split(self, X, y):
        # Need at least two elements to split a node.
        m = y.size
        if m <= self.min_split:
            return None, None

        # Count of each class in the current node.
        num_of_parent = [np.sum(y == c) for c in range(self.n_of_classes_)]

        # Gini of current node.
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_of_parent)
        best_idx, best_thr = None, None

        # Loop through all features.
        for idx in range(self.n_of_features_):
            # Sort data along selected feature.
            thresholds, classes = zip(*sorted(zip(X.iloc[:, idx], y)))
            num_left = [0] * self.n_of_classes_
            num_right = num_of_parent.copy()
            for i in range(1, m):  # possible split positions
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_of_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_of_classes_)
                )

                # The Gini impurity of a split is the weighted average of the Gini
                # impurity of the children.
                gini = (i * gini_left + (m - i) * gini_right) / m

                if thresholds[i] == thresholds[i - 1]:
                    continue

                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2 # middle point
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        """Build a decision tree by recursively finding the best split."""
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_of_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(
            gini=self._gini(y),
            num_samples=y.size,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )

        # Split recursively until maximum depth is reached.
        if depth < self.max_depth:
            idx, thr = self._split(X, y)
            if idx is not None:
                indices_left = X.query(f'{X.columns[idx]} < {thr}').index.tolist()
                if len(X.index) - len(indices_left) < self.min_leaf:
                    indices_left = X.index.tolist()
                elif len(indices_left) < self.min_leaf:
                    indeces_left = []
                X_left, y_left = X.loc[indices_left], y.loc[indices_left]
                X_right, y_right = X.drop(indices_left, axis=0), y.drop(indices_left, axis=0)
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs, features):
        """Predict class for a single sample."""
        node = self.tree_full
        while node.left:
            if inputs[features[node.feature_index]] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

Above is an implementation of decision tree to which we can set depth, minimal split parameter and minimal leaf parameter
if there are less items in node than min_split the node will no longer split and its result will be given
if there would be less items in any of child nodes than min_leaf all items will go to the other leaf

In [ ]:
X_KNI_valid_train, X_KNI_valid_test, y_KNI_valid_train, y_KNI_valid_test = train_test_split(X_KNI_valid, y_KNI_valid, test_size=0.33, random_state=40)
# Fit data.
clf_one_KNI = OwnTreeImplementation(max_depth=1, min_split=1, min_leaf=1)
clf_one_KNI.fit(X_KNI_valid_train, y_KNI_valid_train)
y_KNI_valid_pred = clf_one_KNI.predict(X_KNI_valid_test)
cm_KNI_valid = metrics.confusion_matrix(y_KNI_valid_test, y_KNI_valid_pred)
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


print('Own tree Accuracy:', metrics.accuracy_score(y_KNI_valid_test, y_KNI_valid_pred))
print("Own tree:")
print(pd.DataFrame(cm_KNI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


fpr_KNI_valid, tpr_KNI_valid, threshold_KNI_valid = metrics.roc_curve(y_KNI_valid_test, y_KNI_valid_pred)

auc_KNI_valid = metrics.roc_auc_score(y_KNI_valid_test, y_KNI_valid_pred)
print('FPR', fpr_KNI_valid, 
      '\nTPR', tpr_KNI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_KNI_valid)

plt.plot(fpr_KNI_valid, tpr_KNI_valid, label="data 1, auc="+str(auc_KNI_valid) )
plt.legend(loc=4)
plt.show()

clf_one_KNI.debug(X_KNI_valid.columns, ['negative', 'positive']) # output can be found in OwnKNITreeDepth1.txt

In [ ]:
X_KNI_valid_train, X_KNI_valid_test, y_KNI_valid_train, y_KNI_valid_test = train_test_split(X_KNI_valid, y_KNI_valid, test_size=0.33, random_state=40)
# Fit data.
clf_five_KNI = OwnTreeImplementation(max_depth=5, min_split=1, min_leaf=1)
clf_five_KNI.fit(X_KNI_valid_train, y_KNI_valid_train)
y_KNI_valid_pred = clf_five_KNI.predict(X_KNI_valid_test)
cm_KNI_valid = metrics.confusion_matrix(y_KNI_valid_test, y_KNI_valid_pred)
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


print('Own tree Accuracy:', metrics.accuracy_score(y_KNI_valid_test, y_KNI_valid_pred))
print("Own tree:")
print(pd.DataFrame(cm_KNI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_KNI_valid, fp_KNI_valid, fn_KNI_valid, tp_KNI_valid = cm_KNI_valid.ravel()
print(f'True Positives: {tp_KNI_valid}', 
      f'False Positives: {fp_KNI_valid}', 
      f'True Negatives: {tn_KNI_valid}',
      f'False Negatives: {fn_KNI_valid}', '\n')


fpr_KNI_valid, tpr_KNI_valid, threshold_KNI_valid = metrics.roc_curve(y_KNI_valid_test, y_KNI_valid_pred)

auc_KNI_valid = metrics.roc_auc_score(y_KNI_valid_test, y_KNI_valid_pred)
print('FPR', fpr_KNI_valid, 
      '\nTPR', tpr_KNI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_KNI_valid)

plt.plot(fpr_KNI_valid, tpr_KNI_valid, label="data 1, auc="+str(auc_KNI_valid) )
plt.legend(loc=4)
plt.show()

clf_five_KNI.debug(X_KNI_valid.columns, ['negative', 'positive']) # output can be found in OwnKNITreeDepth5.txt

In [ ]:
X_SI_valid_train, X_SI_valid_test, y_SI_valid_train, y_SI_valid_test = train_test_split(X_SI_valid, y_SI_valid, test_size=0.33, random_state=40)
# Fit data.
clf_one_SI = OwnTreeImplementation(max_depth=1, min_split=1, min_leaf=1)
clf_one_SI.fit(X_SI_valid_train, y_SI_valid_train)

y_SI_valid_pred = clf_one_SI.predict(X_SI_valid_test)
cm_SI_valid = metrics.confusion_matrix(y_SI_valid_test, y_SI_valid_pred)
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')


print('own tree Accuracy:', metrics.accuracy_score(y_SI_valid_test, y_SI_valid_pred))
print("own tree regresion:")
print(pd.DataFrame(cm_SI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')

fpr_SI_valid, tpr_SI_valid, threshold_SI_valid = metrics.roc_curve(y_SI_valid_test, y_SI_valid_pred)

auc_SI_valid = metrics.roc_auc_score(y_SI_valid_test, y_SI_valid_pred)
print('FPR', fpr_SI_valid, 
      '\nTPR', tpr_SI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_SI_valid)

plt.plot(fpr_SI_valid, tpr_SI_valid, label="data 1, auc="+str(auc_SI_valid) )
plt.legend(loc=4)
plt.show()

clf_one_SI.debug(X_KNI_valid.columns, ['negative', 'positive']) # output can be found in OwnSITreeDepth1.txt

In [ ]:
X_SI_valid_train, X_SI_valid_test, y_SI_valid_train, y_SI_valid_test = train_test_split(X_SI_valid, y_SI_valid, test_size=0.33, random_state=40)
# Fit data.
clf_five_SI = OwnTreeImplementation(max_depth=5, min_split=1, min_leaf=1)
clf_five_SI.fit(X_SI_valid_train, y_SI_valid_train)

y_SI_valid_pred = clf_five_SI.predict(X_SI_valid_test)
cm_SI_valid = metrics.confusion_matrix(y_SI_valid_test, y_SI_valid_pred)
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')


print('own tree Accuracy:', metrics.accuracy_score(y_SI_valid_test, y_SI_valid_pred))
print("own tree regresion:")
print(pd.DataFrame(cm_SI_valid, 
                   columns=['Predicted Negative', 'Predicted Positive'], 
                   index=['Actual Negative', 'Actual Positive']), '\n')
tn_SI_valid, fp_SI_valid, fn_SI_valid, tp_SI_valid = cm_SI_valid.ravel()
print(f'True Positives: {tp_SI_valid}', 
      f'False Positives: {fp_SI_valid}', 
      f'True Negatives: {tn_SI_valid}',
      f'False Negatives: {fn_SI_valid}', '\n')

fpr_SI_valid, tpr_SI_valid, threshold_SI_valid = metrics.roc_curve(y_SI_valid_test, y_SI_valid_pred)

auc_SI_valid = metrics.roc_auc_score(y_SI_valid_test, y_SI_valid_pred)
print('FPR', fpr_SI_valid, 
      '\nTPR', tpr_SI_valid, 
      '\nthreshold', 
      '\nthreshold',  
      '\nROC-AUC', auc_SI_valid)

plt.plot(fpr_SI_valid, tpr_SI_valid, label="data 1, auc="+str(auc_SI_valid) )
plt.legend(loc=4)
plt.show()

clf_five_SI.debug(X_KNI_valid.columns, ['negative', 'positive']) # output can be found in OwnSITreeDepth5.txt

# Optimalizacia hyper parametrov

Hyper parametre mozu mat vplyc na vysledok natrenovaneho modelu na to aby sme dosiahli co najlepsie vysledky skusime nastavit rozne kombinacie hyperparametrov pomocou funkcii randomizedSearchCV a nasledne GridSearchCV ktore vyskusalu a cross validuju rozne hyperparametre a vyberu taky ktory ma najlepsie vysledky aby sme nasledne mohli natrenovat model s tymito parametrami

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier

bestAcuracyClassifier = [None,0]
X_SI_valid_train, X_SI_valid_test, y_SI_valid_train, y_SI_valid_test = train_test_split(X_SI_valid, y_SI_valid, test_size=0.33, random_state=40)
X_KNI_valid_train, X_KNI_valid_test, y_KNI_valid_train, y_KNI_valid_test = train_test_split(X_KNI_valid, y_KNI_valid, test_size=0.33, random_state=40)

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=100, max_features=3),
    AdaBoostClassifier(),
    GradientBoostingClassifier(n_estimators=100, random_state=10),
    BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=10)
]
print('SI dataset:')
for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_SI_valid_train, y_SI_valid_train)
    pred = pipe.predict(X_SI_valid_test)

    falseP = 0
    falseN = 0
    for i in range(len(pred)):
        if pred[i] != y_SI_valid_test.tolist()[i]:
            if pred[i] == 1:
                falseP += 1
            else: 
                falseN += 1

    if  accuracy_score(y_KNI_valid_test, pred) > bestAcuracyClassifier[1]:
        bestAcuracyClassifier[0]= classifier
        bestAcuracyClassifier[1] = accuracy_score(y_SI_valid_test, pred)

    print(classifier)
    print()
    print("accuracy:", accuracy_score(y_SI_valid_test, pred))
    print("precision:", precision_score(y_SI_valid_test, pred, average='micro'))
    print("recall:", recall_score(y_SI_valid_test, pred, average='macro'))
    print()
    print(classification_report(y_SI_valid_test, pred))
    print()
    print("False Positive: {}".format(falseP))
    print("False Negative: {}".format(falseN))
    print('-----------------------------------------------------------------------------------------')
print(bestAcuracyClassifier)

In [ ]:
print('KNI dataset:')

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=100, max_features=len(X_KNI_valid_test.columns)),
    AdaBoostClassifier(),
    GradientBoostingClassifier(n_estimators=100, random_state=10),
    BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=10)
]
for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_KNI_valid_train, y_KNI_valid_train)
    pred = pipe.predict(X_KNI_valid_test)

    falseP = 0
    falseN = 0
    for i in range(len(pred)):
        if pred[i] != y_KNI_valid_test.tolist()[i]:
            if pred[i] == 1:
                falseP += 1
            else: 
                falseN += 1

    if  accuracy_score(y_KNI_valid_test, pred) > bestAcuracyClassifier[1]:
        bestAcuracyClassifier[0]= classifier
        bestAcuracyClassifier[1] = accuracy_score(y_KNI_valid_test, pred)

    print(classifier)
    print()
    print("accuracy:", accuracy_score(y_KNI_valid_test, pred))
    print("precision:", precision_score(y_KNI_valid_test, pred, average='micro'))
    print("recall:", recall_score(y_KNI_valid_test, pred, average='macro'))
    print()
    print(classification_report(y_KNI_valid_test, pred))
    print()
    print("False Positive: {}".format(falseP))
    print("False Negative: {}".format(falseN))
    print('-----------------------------------------------------------------------------------------')
print(bestAcuracyClassifier)

In [ ]:
randomForestBase=RandomForestClassifier()

In [ ]:
rf = RandomForestClassifier()

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 10 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_SI_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 10, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

rf_KNI_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 10, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_SI_random.fit(X_SI_valid_train, y_SI_valid_train)

In [ ]:
# Fit the random search model
rf_KNI_random.fit(X_KNI_valid_train, y_KNI_valid_train)

In [ ]:
rf_SI_random.best_params_

In [ ]:
rf_KNI_random.best_params_

In [ ]:
randomForestRandomized=RandomForestClassifier(n_estimators= 400,min_samples_split= 5,min_samples_leaf= 1,max_features= "sqrt",max_depth= 30,bootstrap= True)

In [ ]:

from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
rf = RandomForestClassifier(random_state = 42)

# Instantiate the grid search model
grid_SI_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2, return_train_score=True)
grid_KNI_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
grid_SI_search.fit(X_train, y_train)

In [ ]:
grid_KNI_search.fit(X_train, y_train)

In [ ]:
grid_SI_search.best_params_

In [ ]:
grid_KNI_search.best_params_

In [ ]:
randomForestGrid=RandomForestClassifier(bootstrap=True, max_depth= 80,max_features= 2,min_samples_leaf= 3,min_samples_split= 8,n_estimators= 100)

In [ ]:
ownImplTreeBase = OwnTreeImplementation()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# Minimum number of samples required to split a node
min_split = [2, 5, 10, 15, 20]
# Minimum number of samples required at each leaf node
min_leaf = [1, 2, 4, 6, 10, 12]

# Create the random grid
random_grid = {'max_depth': max_depth,
               'min_split': min_samples_split,
               'min_leaf': min_samples_leaf}

pprint(random_grid)

In [ ]:
best_acc = 0
best_max_depth = 0
best_min_split = 0
best_min_leaf = 0
for depth in random_grid['max_depth']:
    for split in random_grid['min_split']:
        for leaf in random_grid['min_leaf']:
            ot = OwnTreeImplementation(max_depth=depth, min_split=split, min_leaf=leaf)
            ot.fit(X_SI_valid_train, y_SI_valid_train)
            pred = ot.predict(X_SI_valid_test)
            acc = accuracy_score(y_SI_valid_test, pred)
            if acc > best_acc:
                best_acc = acc
                best_max_depth = depth
                best_min_split = split
                best_min_leaf = leaf

print('acc: ', best_acc)
print('max_depth: ', best_max_depth)
print('min_split: ', best_min_split)
print('min_leaf: ', best_min_leaf)

In [ ]:
best_acc = 0
best_max_depth = 0
best_min_split = 0
best_min_leaf = 0
for depth in random_grid['max_depth']:
    for split in random_grid['min_split']:
        for leaf in random_grid['min_leaf']:
            ot = OwnTreeImplementation(max_depth=depth, min_split=split, min_leaf=leaf)
            ot.fit(X_KNI_valid_train, y_KNI_valid_train)
            pred = ot.predict(X_KNI_valid_test)
            acc = accuracy_score(y_KNI_valid_test, pred)
            if acc > best_acc:
                best_acc = acc
                best_max_depth = depth
                best_min_split = split
                best_min_leaf = leaf

print('acc: ', best_acc)
print('max_depth: ', best_max_depth)
print('min_split: ', best_min_split)
print('min_leaf: ', best_min_leaf)

In [ ]:
optimizedOwntreeImplSI = OwnTreeImplementation(max_depth=10, min_leaf=4, min_split=2)

In [ ]:
optimizedOwntreeImplSI.fit(X_SI_valid_train, y_SI_valid_train)
pred=optimizedOwntreeImplSI.predict(X_SI_valid_test)
print("accuracy:", accuracy_score(y_SI_valid_test, pred))
print("precision:", precision_score(y_SI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_SI_valid_test, pred, average='macro'))

In [ ]:
optimizedOwntreeImpl.fit(X_KNI_valid_train, y_KNI_valid_train)
pred=optimizedOwntreeImpl.predict(X_KNI_valid_test)
print("accuracy:", accuracy_score(y_KNI_valid_test, pred))
print("precision:", precision_score(y_KNI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_KNI_valid_test, pred, average='macro'))

In [ ]:
randomForestGrid.fit(X_SI_valid_train, y_SI_valid_train)
pred=randomForestGrid.predict(X_SI_valid_test)
print("accuracy:", accuracy_score(y_SI_valid_test, pred))
print("precision:", precision_score(y_SI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_SI_valid_test, pred, average='macro'))

In [ ]:
randomForestRandomized.fit(X_SI_valid_train, y_SI_valid_train)
pred=randomForestRandomized.predict(X_SI_valid_test)
print("accuracy:", accuracy_score(y_SI_valid_test, pred))
print("precision:", precision_score(y_SI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_SI_valid_test, pred, average='macro'))

In [ ]:
randomForestBase.fit(X_SI_valid_train, y_SI_valid_train)
pred=randomForestBase.predict(X_SI_valid_test)
print("accuracy:", accuracy_score(y_SI_valid_test, pred))
print("precision:", precision_score(y_SI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_SI_valid_test, pred, average='macro'))

In [ ]:
randomForestGrid.fit(X_KNI_valid_train, y_KNI_valid_train)
pred=randomForestGrid.predict(X_KNI_valid_test)
print("accuracy:", accuracy_score(y_KNI_valid_test, pred))
print("precision:", precision_score(y_KNI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_KNI_valid_test, pred, average='macro'))

In [ ]:
randomForestRandomized.fit(X_KNI_valid_train, y_KNI_valid_train)
pred=randomForestRandomized.predict(X_KNI_valid_test)
print("accuracy:", accuracy_score(y_KNI_valid_test, pred))
print("precision:", precision_score(y_KNI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_KNI_valid_test, pred, average='macro'))

In [ ]:
randomForestBase.fit(X_KNI_valid_train, y_KNI_valid_train)
pred=randomForestBase.predict(X_KNI_valid_test)
print("accuracy:", accuracy_score(y_KNI_valid_test, pred))
print("precision:", precision_score(y_KNI_valid_test, pred, average='micro'))
print("recall:", recall_score(y_KNI_valid_test, pred, average='macro'))

Vyssie mame porovnane klasifikatory z kniznice scikit s vlastnou implementaciou a vidime ze nasa vlastna implementacia ma horsiu accuracy ako prepripravene kniznice vidime vsak aj ze nasa vlastna implementacia s hlbkou 1 a teda vyuzitim iba jedneho atributu ma pomerne dobru presnost co naznacuje ze vysledok sa z velmi velkej casti odraza prave od jedneho atributu. zaujimave je ze v pripade Si datasetu bolo tymto atributom std_glucose a pre KNI dataset kurtosis_glucose toto moze byt dosledkom roznych metod nahradenia nanov. Na vyslednej accuracy sa to vsak neodrazilo.

## Vizualizacie jednotlivych klasifikatorov

In [ ]:
# Vizualizacia stromu pre randomForestgrid
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG

graph = Source(export_graphviz(randomForestGrid,
                               out_file=None,
                               feature_names=X_KNI_valid.columns,
                               class_names=["positive", "negative"],
                               filled = True))
display(SVG(graph.pipe(format='svg')))

from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

In [ ]:
# Vizualizacia stromu pre randomForestrandomized
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG

graph = Source(export_graphviz(randomForestRandomized,
                               out_file=None,
                               feature_names=X_KNI_valid.columns,
                               class_names=["positive", "negative"],
                               filled = True))
display(SVG(graph.pipe(format='svg')))

from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

In [ ]:
# Vizualizacia stromu pre randomForestBase
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG

graph = Source(export_graphviz(randomForestBase,
                               out_file=None,
                               feature_names=X_KNI_valid.columns,
                               class_names=["positive", "negative"],
                               filled = True))
display(SVG(graph.pipe(format='svg')))

from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

# Porovnanie zvolenych strategii

Vyuzili sme dve rozne metody doplnenie chybajucich hodnot a to pomocou simple imputeru a funkcie mean a doplnenie K najblizsich susedov

V konecnom dosledku mozeme vidiet ze pre optimalizovane hyperparametre su vysledky pre obe mnoziny rovnake avsak pri SI mame v datasete iba 3 atributy a pri KNI 4 pri viacero atributoch v oboch datasetoch vznikala s narastajucou sa komplexitou narastajuca chyba co znacilo pretrenovanie mozeme teda povazovat dataset SI za menej komplexny a mneje robustny kedze sme ho museli viac zjednodusit aby sme dosiahli nezvysovanie sa chyby s komplexitou.